In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.utils import filtrar_meses, verificar_columnas_y_tipos, estandarizar_nombres_columnas
from src.utils import funciones 
from src.utils import verificar_consistencia_coordenadas

In [ ]:
import pandas as pd

from utils import corregir_longitudes_inconsistentes

def procesar_recorridos_anio(ruta_csv_entrada, ruta_csv_salida, meses_a_eliminar, col_origen="fecha_origen_recorrido", col_destino="fecha_destino_recorrido"):

    df = pd.read_csv(ruta_csv_entrada)

    df_filtrado, filas_eliminadas_origen = filtrar_meses(df, col_origen, meses_a_eliminar)
    print(f"Se eliminaron {filas_eliminadas_origen} filas por fecha de origen.")

    df_filtrado, filas_eliminadas_destino = filtrar_meses(df_filtrado, col_destino, meses_a_eliminar)
    print(f"Se eliminaron {filas_eliminadas_destino} filas por fecha de destino.")

    total_eliminadas = filas_eliminadas_origen + filas_eliminadas_destino

    df_filtrado.to_csv(ruta_csv_salida, index=False)

    return df_filtrado, total_eliminadas

def limpiar_y_guardar_recorridos():
    años_columnas_a_dropear = {
        2020: [],
        2021: ["Género"],
        2022: ["X"],
        2023: []
    }

    for año, columnas in años_columnas_a_dropear.items():
        ruta_entrada = f"data/recorridos/raw/trips_{año}.csv"
        ruta_salida = f"data/recorridos/processed/trips_{año}_pr.csv"
        
        df = pd.read_csv(ruta_entrada, index_col=0)
        
        if columnas:
            df = df.drop(columns=[col for col in columnas if col in df.columns])
        
        df = df.reset_index(drop=True)
        df.to_csv(ruta_salida, index=False)


def forzar_tipos(df, columnas, tipo):
    for col in columnas:
        if col in df.columns:
            df[col] = df[col].astype(tipo)
    return df

def limpiar_y_normalizar_archivos(archivos, funciones):
    anios = [2020, 2021, 2022, 2023]

    estandarizar_nombres_columnas(archivos)

    for anio in [2020, 2023]:
        ruta = f"data/recorridos/processed/trips_{anio}_pr.csv"
        df = pd.read_csv(ruta)
        df = df.dropna()
        df.to_csv(ruta, index=False)

    for anio in anios:
        print(f"Procesando año {anio}...")
        ruta = f"data/recorridos/processed/trips_{anio}_pr.csv"
        df = pd.read_csv(ruta)
        df_limpio = funciones[str(anio)](df)
        df_limpio = forzar_tipos(df_limpio, ["id_usuario"], 'float64')
        df_limpio.to_csv(ruta, index=False)

    verificar_columnas_y_tipos(archivos)

def corregir_coordenadas_con_base_2024(path_2024, archivos_a_corregir):
    df_2024 = pd.read_csv(path_2024)
    coords_2024 = {}

    for rol in ["origen", "destino"]:
        id_col = f"id_estacion_{rol}"
        lat_col = f"lat_estacion_{rol}"
        lon_col = f"long_estacion_{rol}"

        for _, row in df_2024.iterrows():
            est_id = row[id_col]
            lat = row[lat_col]
            lon = row[lon_col]
            if pd.notna(est_id) and pd.notna(lat) and pd.notna(lon):
                coords_2024[est_id] = (lat, lon)

    for anio, path in archivos_a_corregir.items():
        if not os.path.exists(path):
            print(f"⚠️ No se encontró el archivo de {anio}")
            continue

        df = pd.read_csv(path)

        for rol in ["origen", "destino"]:
            id_col = f"id_estacion_{rol}"
            lat_col = f"lat_estacion_{rol}"
            lon_col = f"long_estacion_{rol}"

            def corregir_coord(row):
                est_id = row[id_col]
                if est_id in coords_2024:
                    lat, lon = coords_2024[est_id]
                    row[lat_col] = lat
                    row[lon_col] = lon
                return row

            df = df.apply(corregir_coord, axis=1)

        df.to_csv(path, index=False)
        print(f"✅ Archivo actualizado y sobrescrito: {path}")

from src.utils import corregir_latitud_estacion\

def corregir_estaciones_2020():
    path_2020 = "data/recorridos/processed/trips_2020_pr.csv"
    df_2020 = pd.read_csv(path_2020)
    df_2020 = corregir_latitud_estacion(df_2020, est_id=240)
    df_2020.to_csv(path_2020, index=False)
    corregir_longitudes_inconsistentes(path_2020)

def verificar_coordenadas_todos_los_anios(anios = [], test: bool = False, path_csv: str = None):
    if not test: 
        for anio in anios:
            path = f"data/recorridos/processed/trips_{anio}_pr.csv"
            verificar_consistencia_coordenadas(path, anio=anio)
    if test: 
        verificar_consistencia_coordenadas(path_csv, test=True)


import pandas as pd
import os

def construir_diccionario_estaciones(paths_descendentes):
    estaciones_dict = {}

    for path in paths_descendentes:
        if not os.path.exists(path):
            print(f"⚠️ Archivo no encontrado: {path}")
            continue

        df = pd.read_csv(path)

        for rol in ["origen", "destino"]:
            id_col = f"id_estacion_{rol}"
            lat_col = f"lat_estacion_{rol}"
            lon_col = f"long_estacion_{rol}"

            for _, row in df.iterrows():
                est_id = row[id_col]
                lat = row[lat_col]
                lon = row[lon_col]

                if pd.notna(est_id) and pd.notna(lat) and pd.notna(lon):
                    if est_id not in estaciones_dict:
                        estaciones_dict[est_id] = (lat, lon)

    return estaciones_dict

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

def asignar_barrios_a_datasets(df_estaciones, anios=None, test=False, path_test=None, path_shapefile="data/barrios/barrios.shp"):
    """
    Enriquece datasets de viajes agregando los barrios de origen y destino según las estaciones.

    Parámetros:
    - df_estaciones: DataFrame con columnas ['id_estacion', 'lat', 'lon']
    - anios: lista de años a procesar (si test=False)
    - test: si es True, se aplica solo al archivo en path_test
    - path_test: path a un CSV de viajes para modo test
    - path_shapefile: ruta al archivo de polígonos de barrios
    """
    # ========================
    # 📍 1. Cargar shapefile de barrios
    # ========================
    barrios_gdf = gpd.read_file(path_shapefile)
    if "nombre" in barrios_gdf.columns and "barrio" not in barrios_gdf.columns:
        barrios_gdf = barrios_gdf.rename(columns={"nombre": "barrio"})

    # ========================
    # 📍 2. Join espacial: estaciones → barrios
    # ========================
    df_estaciones["geometry"] = df_estaciones.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
    estaciones_gdf = gpd.GeoDataFrame(df_estaciones, geometry="geometry", crs=barrios_gdf.crs)

    estaciones_con_barrios = gpd.sjoin(estaciones_gdf, barrios_gdf, how="left", predicate="within")

    # Guardar CSV intermedio (opcional)
    estaciones_con_barrios[["id_estacion", "lat", "lon", "barrio"]].to_csv("data/estaciones_con_barrios.csv", index=False)

    # Diccionario id_estacion → barrio
    mapa_barrio = estaciones_con_barrios.set_index("id_estacion")["barrio"].to_dict()

    # Correcciones manuales
    mapa_barrio[111] = "PUERTO MADERO"
    mapa_barrio[541] = "PALERMO"

    # ========================
    # 📍 3. Enriquecer datasets
    # ========================
    if test:
        if path_test is None or not os.path.exists(path_test):
            print("⚠️ Path inválido o no encontrado para test.")
            return
        df = pd.read_csv(path_test)
        df["barrio_origen"] = df["id_estacion_origen"].map(mapa_barrio)
        df["barrio_destino"] = df["id_estacion_destino"].map(mapa_barrio)
        df.to_csv(path_test, index=False)
        print(f"🧪 Archivo enriquecido (modo test): {path_test}")
        return

    if anios is None:
        print("⚠️ No se especificaron años para procesar.")
        return

    for anio in anios:
        path = f"data/recorridos/processed/trips_{anio}_pr.csv"
        if not os.path.exists(path):
            print(f"⚠️ Archivo no encontrado: {path}")
            continue

        df = pd.read_csv(path)
        df["barrio_origen"] = df["id_estacion_origen"].map(mapa_barrio)
        df["barrio_destino"] = df["id_estacion_destino"].map(mapa_barrio)
        df.to_csv(path, index=False)
        print(f"✅ Enriquecido con barrios: {anio}")


import pandas as pd
import os

def obtener_o_construir_df_estaciones(lista_paths, path_guardado="data/estaciones_unico.csv"):
    if os.path.exists(path_guardado):
        print("📂 Usando df_estaciones ya guardado.")
        return pd.read_csv(path_guardado)

    print("⚙️ Construyendo df_estaciones desde archivos...")
    estaciones_dict = {}

    for path in lista_paths:
        if not os.path.exists(path):
            print(f"⚠️ Archivo no encontrado: {path}")
            continue

        df = pd.read_csv(path)

        for rol in ["origen", "destino"]:
            id_col = f"id_estacion_{rol}"
            lat_col = f"lat_estacion_{rol}"
            lon_col = f"long_estacion_{rol}"

            for _, row in df.iterrows():
                est_id = row[id_col]
                lat = row[lat_col]
                lon = row[lon_col]

                if pd.notna(est_id) and pd.notna(lat) and pd.notna(lon):
                    if est_id not in estaciones_dict:
                        estaciones_dict[est_id] = (lat, lon)

    df_estaciones = pd.DataFrame([
        {"id_estacion": est_id, "lat": lat, "lon": lon}
        for est_id, (lat, lon) in estaciones_dict.items()
    ])

    # Guardar el CSV para futuros usos
    os.makedirs(os.path.dirname(path_guardado), exist_ok=True)
    df_estaciones.to_csv(path_guardado, index=False)
    print(f"✅ df_estaciones guardado en {path_guardado}")

    return df_estaciones

def analyze_barrios(path_shapefile="data/barrios/barrios.shp"):
    """
    Analiza y muestra información general del shapefile de barrios.
    
    Parámetros:
    - path_shapefile: ruta al archivo .shp
    
    Salida:
    - Imprime columnas, cantidad de barrios únicos y sus nombres.
    """
    try:
        barrios_gdf = gpd.read_file(path_shapefile)
    except Exception as e:
        print(f"❌ Error al leer el shapefile: {e}")
        return

    print("🧩 Columnas disponibles en el shapefile:")
    print(barrios_gdf.columns.tolist())

    if "nombre" not in barrios_gdf.columns:
        print("⚠️ La columna 'nombre' no existe en el shapefile.")
        return

    barrios_unicos = sorted(barrios_gdf["nombre"].dropna().unique())
    total = len(barrios_unicos)

    print(f"\n📊 Cantidad de barrios únicos en el shapefile: {total}")
    print("\n🗺️ Barrios:")
    print(barrios_unicos)
    print(f"\n🧮 Total de barrios: {total}")


def analizar_presencia_estaciones(archivos, anios):
    estaciones_por_anio = {}

    for archivo, anio in zip(archivos, anios):
        if not os.path.exists(archivo):
            print(f"⚠️ Archivo no encontrado para el año {anio}")
            continue

        df = pd.read_csv(archivo)
        estaciones_origen = df['id_estacion_origen'].dropna().unique()
        estaciones_destino = df['id_estacion_destino'].dropna().unique()
        estaciones = set(estaciones_origen).union(set(estaciones_destino))
        estaciones_por_anio[anio] = estaciones

    todas_las_estaciones = sorted(set.union(*estaciones_por_anio.values()))
    tabla_presencia = pd.DataFrame(index=todas_las_estaciones)

    for anio in anios:
        estaciones = estaciones_por_anio.get(anio, set())
        tabla_presencia[anio] = ["✓" if est in estaciones else "" for est in tabla_presencia.index]

    tabla_presencia["Anios_presente"] = tabla_presencia[anios].apply(lambda row: sum(cell == "✓" for cell in row), axis=1)
    tabla_presencia = tabla_presencia.sort_index()

    print("\n📋 Presencia de estaciones por año (por ID, ordenadas por cantidad de años presentes):")
    print(tabla_presencia)

    print("\n📈 Cantidad de estaciones por año:")
    for anio in anios:
        cantidad = len(estaciones_por_anio.get(anio, set()))
        print(f"  - {anio}: {cantidad} estaciones")

    return tabla_presencia, estaciones_por_anio


def mostrar_estaciones_faltantes_en_2024(estaciones_por_anio):
    print("\n🔍 Estaciones que aparecían en años anteriores pero NO en 2024:")

    estaciones_2024 = estaciones_por_anio.get(2024, set())

    for anio in [2020, 2021, 2022, 2023]:
        estaciones_anio = estaciones_por_anio.get(anio, set())
        solo_en_anio = sorted(estaciones_anio - estaciones_2024)
        print(f"\n➡️ Estaciones en {anio} pero NO en 2024 ({len(solo_en_anio)}):")
        print(solo_en_anio)

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from matplotlib.lines import Line2D
import os

def estaciones_graf():
    # ----------------------
    # COLORES HARDCODEADOS
    # ----------------------
    colores_barrios = [
        "red", "green", "blue", "darkorange", "purple", "cyan", "magenta", "gold",
        "orchid", "tomato", "lightpink", "deepskyblue", "chartreuse", "firebrick", "sienna", "dodgerblue",
        "plum", "turquoise", "slateblue", "darkgreen", "hotpink", "indigo", "salmon", "navy",
        "darkred", "chocolate", "crimson", "teal", "coral", "darkviolet", "mediumorchid", "mediumseagreen",
        "darkmagenta", "mediumblue", "olivedrab", "slategray", "seagreen", "deeppink", "steelblue", "brown",
        "orangered", "forestgreen", "darkcyan", "violet", "palevioletred", "blueviolet", "darkslateblue", "limegreen"
    ]

    # ----------------------
    # CARGAR BARRIOS
    # ----------------------
    barrios_gdf = gpd.read_file("data/barrios/barrios.shp").to_crs(epsg=4326)
    barrios_unicos = sorted(barrios_gdf["nombre"].unique())
    assert len(colores_barrios) == len(barrios_unicos), "La cantidad de colores no coincide con la de barrios"
    color_dict = {barrio: colores_barrios[i] for i, barrio in enumerate(barrios_unicos)}

    # ----------------------
    # CARGAR ESTACIONES DE TODOS LOS AÑOS
    # ----------------------
    archivos = [
        "data/recorridos/processed/trips_2024_pr.csv",
        "data/recorridos/processed/trips_2023_pr.csv",
        "data/recorridos/processed/trips_2022_pr.csv",
        "data/recorridos/processed/trips_2021_pr.csv",
        "data/recorridos/processed/trips_2020_pr.csv"
    ]

    estaciones_dict = {}
    for path in archivos:
        if not os.path.exists(path):
            continue
        df = pd.read_csv(path)
        for rol in ["origen", "destino"]:
            for _, row in df.iterrows():
                est_id = row[f"id_estacion_{rol}"]
                lat = row[f"lat_estacion_{rol}"]
                lon = row[f"long_estacion_{rol}"]
                if pd.notna(est_id) and pd.notna(lat) and pd.notna(lon):
                    if est_id not in estaciones_dict:
                        estaciones_dict[est_id] = (lat, lon)

    df_estaciones_todas = pd.DataFrame([
        {"id_estacion": est_id, "lat": lat, "lon": lon}
        for est_id, (lat, lon) in estaciones_dict.items()
    ])
    df_estaciones_todas["geometry"] = df_estaciones_todas.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
    gdf_estaciones_todas = gpd.GeoDataFrame(df_estaciones_todas, geometry="geometry", crs="EPSG:4326")

    # ----------------------
    # CARGAR ESTACIONES 2024
    # ----------------------
    estaciones_2024 = {}
    df_2024 = pd.read_csv("data/recorridos/processed/trips_2024_pr.csv")
    for rol in ["origen", "destino"]:
        for _, row in df_2024.iterrows():
            est_id = row[f"id_estacion_{rol}"]
            lat = row[f"lat_estacion_{rol}"]
            lon = row[f"long_estacion_{rol}"]
            if pd.notna(est_id) and pd.notna(lat) and pd.notna(lon):
                if est_id not in estaciones_2024:
                    estaciones_2024[est_id] = (lat, lon)

    df_estaciones_2024 = pd.DataFrame([
        {"id_estacion": est_id, "lat": lat, "lon": lon}
        for est_id, (lat, lon) in estaciones_2024.items()
    ])
    df_estaciones_2024["geometry"] = df_estaciones_2024.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
    gdf_estaciones_2024 = gpd.GeoDataFrame(df_estaciones_2024, geometry="geometry", crs="EPSG:4326")

    # ----------------------
    # PLOT
    # ----------------------
    fig, axes = plt.subplots(1, 2, figsize=(20, 12))
    ax1, ax2 = axes

    for barrio, shape in barrios_gdf.groupby("nombre"):
        shape.plot(ax=ax1, color=color_dict[barrio], edgecolor="black", linewidth=0.5)
    gdf_estaciones_todas.plot(ax=ax1, color="black", edgecolor="white", markersize=40, alpha=0.9)
    ax1.set_title("📍 Todas las estaciones (2020–2024)", fontsize=18)
    ax1.set_xlabel("Longitud", fontsize=13)
    ax1.set_ylabel("Latitud", fontsize=13)
    ax1.axis("equal")
    ax1.grid(True)

    for barrio, shape in barrios_gdf.groupby("nombre"):
        shape.plot(ax=ax2, color=color_dict[barrio], edgecolor="black", linewidth=0.5)
    gdf_estaciones_2024.plot(ax=ax2, color="black", edgecolor="white", markersize=40, alpha=0.9)
    ax2.set_title("📍 Estaciones activas en 2024", fontsize=18)
    ax2.set_xlabel("Longitud", fontsize=13)
    ax2.set_ylabel("Latitud", fontsize=13)
    ax2.axis("equal")
    ax2.grid(True)

    handles = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor=color_dict[barrio],
               markeredgecolor='black', markersize=10, label=barrio)
        for barrio in barrios_unicos
    ]
    fig.legend(handles=handles, title="Barrios", loc="center left", bbox_to_anchor=(1.01, 0.5), fontsize="small")

    plt.tight_layout()
    plt.show()

import pandas as pd
import os

def analisis_usuarios(test=False, path_test=None):
    anios_recorridos = [2020, 2021, 2022, 2023, 2024]
    anios_usuarios_nuevos = [2015, 2016, 2017, 2018, 2019]

    # === 1. Usuarios de recorridos
    if test:
        if not path_test or not os.path.exists(path_test):
            print("⚠️ Path de test inválido o no encontrado.")
            return
        print(f"🧪 Modo test: analizando usuarios en {path_test}")
        df_usuarios_recorridos = pd.read_csv(path_test, usecols=["id_usuario"])
        df_usuarios_recorridos = df_usuarios_recorridos.dropna().astype({"id_usuario": int})
        df_usuarios_recorridos["año_recorrido"] = "TEST"
    else:
        usuarios_recorridos = []
        for anio in anios_recorridos:
            path = f"data/recorridos/processed/trips_{anio}_pr.csv"
            if os.path.exists(path):
                df = pd.read_csv(path, usecols=["id_usuario"])
                df = df.dropna().astype({"id_usuario": int})
                df["año_recorrido"] = anio
                usuarios_recorridos.append(df)
            else:
                print(f"⚠️ No se encontró archivo de recorridos {anio}")
        df_usuarios_recorridos = pd.concat(usuarios_recorridos, ignore_index=True)

    # === 2. Usuarios registrados 2020–2024
    usuarios_registrados = []
    for anio in anios_recorridos:
        path = f"data/usuarios/processed/usuarios_ecobici_{anio}_limpio.csv"
        if os.path.exists(path):
            df = pd.read_csv(path, usecols=["ID_usuario"])
            df = df.dropna().astype({"ID_usuario": int})
            usuarios_registrados.append(df)
        else:
            print(f"⚠️ No se encontró archivo de usuarios {anio}")
    df_usuarios_registrados = pd.concat(usuarios_registrados, ignore_index=True)
    usuarios_2020_2024_set = set(df_usuarios_registrados["ID_usuario"].unique())

    # === 3. Usuarios no registrados en 2020–2024
    df_no_registrados = df_usuarios_recorridos[
        ~df_usuarios_recorridos["id_usuario"].isin(usuarios_2020_2024_set)
    ].copy()

    # === 4. Usuarios registrados 2015–2019
    usuarios_extra = []
    for anio in anios_usuarios_nuevos:
        path = f"data/new_data/processed/usuarios_ecobici_{anio}_limpio.csv"
        if os.path.exists(path):
            df = pd.read_csv(path, dtype=str)
            df.columns = [col.replace('"', '') for col in df.columns]
            df = df.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)
            df["ID_usuario"] = df["ID_usuario"].astype(int)
            df["año_archivo"] = anio
            usuarios_extra.append(df[["ID_usuario", "año_archivo"]])
        else:
            print(f"⚠️ No se encontró archivo de usuarios nuevos {anio}")
    df_usuarios_extra = pd.concat(usuarios_extra, ignore_index=True)
    usuarios_2015_2019_set = set(df_usuarios_extra["ID_usuario"].unique())

    # === 5. Análisis
    if test:
        usuarios_en_test = df_no_registrados["id_usuario"].unique()
        total = len(usuarios_en_test)
        encontrados = sum(uid in usuarios_2015_2019_set for uid in usuarios_en_test)
        no_encontrados = total - encontrados
        print("\n🔍 Resultados (modo test):")
        print(f"Usuarios en test no registrados en 2020–2024: {total}")
        print(f"Encontrados en 2015–2019: {encontrados}")
        print(f"Siguen sin aparecer: {no_encontrados}")
    else:
        resultados = []
        for anio in sorted(df_no_registrados["año_recorrido"].unique()):
            usuarios_en_anio = df_no_registrados[df_no_registrados["año_recorrido"] == anio]["id_usuario"].unique()
            total = len(usuarios_en_anio)
            encontrados = sum(uid in usuarios_2015_2019_set for uid in usuarios_en_anio)
            no_encontrados = total - encontrados
            resultados.append({
                "año_recorrido": anio,
                "usuarios_faltantes_2020_2024": total,
                "encontrados_en_2015_2019": encontrados,
                "siguen_sin_aparecer": no_encontrados
            })

        df_resultado = pd.DataFrame(resultados)
        print("🔍 Usuarios de recorridos no encontrados en 2020–2024, pero sí en 2015–2019:")
        print(df_resultado)
    # === 6. Filas generadas por usuarios totalmente desconocidos
    usuarios_totalmente_desconocidos = set(df_no_registrados["id_usuario"]) - usuarios_2015_2019_set

    df_filas_desconocidas = df_usuarios_recorridos[
        df_usuarios_recorridos["id_usuario"].isin(usuarios_totalmente_desconocidos)
    ]

    if test:
        print(f"\n📊 Filas generadas por usuarios totalmente desconocidos (ni 2015–2019 ni 2020–2024): {len(df_filas_desconocidas)}")
    else:
        conteo_filas_por_anio = (
            df_filas_desconocidas.groupby("año_recorrido")
            .size()
            .reset_index(name="filas_totales_usuarios_desconocidos")
        )
        print("\n📊 Filas generadas por usuarios que no aparecen en ningún archivo (ni 2015–2019 ni 2020–2024):")
        print(conteo_filas_por_anio)

# SUPER FUNC

In [ ]:
ruta_entrada = "data/recorridos/raw/trips_2024.csv"
ruta_salida = "data/recorridos/processed/trips_2024_pr.csv"
meses = [9, 10, 11, 12]

df_final, total = procesar_recorridos_anio(ruta_entrada, ruta_salida, meses)

limpiar_y_guardar_recorridos()

archivos = [
    "data/recorridos/processed/trips_2020_pr.csv",
    "data/recorridos/processed/trips_2021_pr.csv",
    "data/recorridos/processed/trips_2022_pr.csv",
    "data/recorridos/processed/trips_2023_pr.csv",
    "data/recorridos/processed/trips_2024_pr.csv",
]

limpiar_y_normalizar_archivos(archivos, funciones)

archivos_corregir = {
    2020: "data/recorridos/processed/trips_2020_pr.csv",
    2021: "data/recorridos/processed/trips_2021_pr.csv",
    2022: "data/recorridos/processed/trips_2022_pr.csv",
    2023: "data/recorridos/processed/trips_2023_pr.csv",
}

tabla, estaciones_por_anio = analizar_presencia_estaciones(archivos, [2020, 2021, 2022, 2023, 2024]])
mostrar_estaciones_faltantes_en_2024(estaciones_por_anio)

verificar_consistencia_coordenadas("data/recorridos/processed/trips_2024_pr.csv", anio=2024)
corregir_coordenadas_con_base_2024("data/recorridos/processed/trips_2024_pr.csv", archivos_corregir)
from src.utils import corregir_longitudes_inconsistentes
corregir_longitudes_inconsistentes("data/recorridos/processed/trips_2021_pr.csv")
corregir_estaciones_2020()
verificar_coordenadas_todos_los_anios([2020, 2021, 2022, 2023])
archivos = [
    "data/recorridos/processed/trips_2024_pr.csv",
    "data/recorridos/processed/trips_2023_pr.csv",
    "data/recorridos/processed/trips_2022_pr.csv",
    "data/recorridos/processed/trips_2021_pr.csv",
    "data/recorridos/processed/trips_2020_pr.csv"
]

analyze_barrios()
diccionario = construir_diccionario_estaciones(archivos)
print(f"✅ Diccionario construido con {len(diccionario)} estaciones.")
df_estaciones = obtener_o_construir_df_estaciones(archivos)
asignar_barrios_a_datasets(df_estaciones, anios=[2020, 2021, 2022, 2023, 2024])
estaciones_graf()
analisis_usuarios()

In [ ]:
#esto para test

verificar_coordenadas_todos_los_anios(test = True, path_csv="pathdeltest")
df_estaciones = obtener_o_construir_df_estaciones(None)
asignar_barrios_a_datasets(df_estaciones, test=True, path_test="pathtest")
analisis_usuarios(test=True, path_test="pathtest")

# ARMADO DS

In [ ]:
import pandas as pd
import os

def insumos_modelado(test=False, path_test=None):
    """
    Crea el dataset modelado, funcionando para modo completo (2020–2024) o test individual.
    """
    # === CARGA DE VIAJES
    if test:
        if not path_test or not os.path.exists(path_test):
            raise FileNotFoundError("❌ Path de test inválido.")
        print(f"🧪 Modo test: usando archivo {path_test}")
        df_viajes = pd.read_csv(path_test)
    else:
        print("📥 Cargando viajes completos 2020–2024...")
        recorridos_dir = "data/recorridos/processed"
        df_viajes = pd.concat([
            pd.read_csv(os.path.join(recorridos_dir, f))
            for f in sorted(os.listdir(recorridos_dir))
            if f.endswith(".csv")
        ], ignore_index=True)

    # === CARGA DE USUARIOS (siempre todos)
    print("👤 Cargando todos los usuarios (2015–2024)...")
    usuarios_dir_1 = "data/usuarios/processed"
    usuarios_dir_2 = "data/new_data/processed"
    archivos_usuarios = []

    for carpeta in [usuarios_dir_1, usuarios_dir_2]:
        archivos_usuarios += [
            os.path.join(carpeta, f)
            for f in sorted(os.listdir(carpeta))
            if f.endswith(".csv")
        ]

    df_usuarios = pd.concat([
        pd.read_csv(f) for f in archivos_usuarios
    ], ignore_index=True)

    # === CARGA DE ESTACIONES (con barrios ya asignados)
    df_estaciones = pd.read_csv("data/estaciones_con_barrios.csv")

    # === LLAMADA A LA FUNCIÓN PRINCIPAL
    from modeling import construir_dataset_modelado_v2
    df_modelado = construir_dataset_modelado_v2(df_viajes, df_usuarios, df_estaciones, test=test)

    return df_modelado

def procesar_columnas_modelado(df_modelado):
    # === Estación del año → int
    mapa_estaciones = {"verano": 1, "otono": 2, "invierno": 3, "primavera": 4}
    df_modelado["estacion_del_anio"] = df_modelado["estacion_del_anio"].map(mapa_estaciones).astype(int)

    # === Conversiones de fecha
    df_modelado["fecha_origen_recorrido"] = pd.to_datetime(df_modelado["fecha_origen_recorrido"], errors="coerce")
    df_modelado["fecha_destino_recorrido"] = pd.to_datetime(df_modelado["fecha_destino_recorrido"], errors="coerce")
    df_modelado["fecha_intervalo"] = pd.to_datetime(df_modelado["fecha_intervalo"], errors="coerce")

    # === Columnas temporales origen
    df_modelado["año_origen"] = df_modelado["fecha_origen_recorrido"].dt.year
    df_modelado["mes_origen"] = df_modelado["fecha_origen_recorrido"].dt.month
    df_modelado["dia_origen"] = df_modelado["fecha_origen_recorrido"].dt.day
    df_modelado["hora_origen"] = df_modelado["fecha_origen_recorrido"].dt.hour
    df_modelado["minuto_origen"] = df_modelado["fecha_origen_recorrido"].dt.minute
    df_modelado["segundo_origen"] = df_modelado["fecha_origen_recorrido"].dt.second

    # === Columnas temporales destino
    df_modelado["año_destino"] = df_modelado["fecha_destino_recorrido"].dt.year
    df_modelado["mes_destino"] = df_modelado["fecha_destino_recorrido"].dt.month
    df_modelado["dia_destino"] = df_modelado["fecha_destino_recorrido"].dt.day
    df_modelado["hora_destino"] = df_modelado["fecha_destino_recorrido"].dt.hour
    df_modelado["minuto_destino"] = df_modelado["fecha_destino_recorrido"].dt.minute
    df_modelado["segundo_destino"] = df_modelado["fecha_destino_recorrido"].dt.second

    # === Columnas temporales intervalo
    df_modelado["año_intervalo"] = df_modelado["fecha_intervalo"].dt.year
    df_modelado["mes_intervalo"] = df_modelado["fecha_intervalo"].dt.month
    df_modelado["dia_intervalo"] = df_modelado["fecha_intervalo"].dt.day
    df_modelado["hora_intervalo"] = df_modelado["fecha_intervalo"].dt.hour
    df_modelado["minuto_intervalo"] = df_modelado["fecha_intervalo"].dt.minute

    # === Edad
    df_modelado["edad_usuario"] = pd.to_numeric(df_modelado["edad_usuario"], errors="coerce").fillna(-1).astype(int)

    # === Mapeo de barrios únicos
    barrios_unicos = pd.unique(df_modelado[["barrio_origen", "barrio_destino"]].values.ravel())
    mapa_barrio = {barrio: i for i, barrio in enumerate(barrios_unicos, start=1)}

    # Guardar el mapa
    with open("data/modelado/mapa_barrio.json", "w") as f:
        json.dump(mapa_barrio, f, ensure_ascii=False)

    df_modelado["barrio_origen"] = df_modelado["barrio_origen"].map(mapa_barrio)
    df_modelado["barrio_destino"] = df_modelado["barrio_destino"].map(mapa_barrio)

    # === Modelo de bicicleta
    df_modelado["modelo_bicicleta"] = df_modelado["modelo_bicicleta"].map({"ICONIC": 1, "FIT": 0}).astype(int)

    # === Eliminar columnas no necesarias
    df_modelado.drop(columns=[
        "hora_dia",
        "fecha_origen_recorrido",
        "fecha_destino_recorrido",
        "fecha_intervalo"
    ], errors="ignore", inplace=True)

    return df_modelado

def generar_atributos_estaciones(
    path_estaciones: str = "data/new_data/raw/nuevas-estaciones-bicicletas-publicas.csv",
    path_ciclovias: str = "data/new_data/raw/ciclovias.geojson",
    output_path: str = "data/new_data/processed/atributos_estaciones.csv"
) -> None:
    import geopandas as gpd
    import pandas as pd
    import os
    from shapely.geometry import Point, LineString, MultiLineString

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Cargar estaciones y ciclovías
    df_est = pd.read_csv(path_estaciones)
    gdf_ciclovias = gpd.read_file(path_ciclovias)

    # Convertir estaciones a GeoDataFrame
    gdf_est = gpd.GeoDataFrame(
        df_est,
        geometry=gpd.points_from_xy(df_est["longitud"], df_est["latitud"]),
        crs="EPSG:4326"
    ).to_crs(epsg=3857)

    # Ciclovías a misma proyección
    gdf_ciclovias = gdf_ciclovias.to_crs(epsg=3857)
    ciclovias_list = list(gdf_ciclovias.geometry)
    ciclovias_union = gdf_ciclovias.unary_union

    # Distancia a la ciclovía más cercana
    gdf_est["dist_ciclovia_m"] = gdf_est.geometry.apply(lambda p: p.distance(ciclovias_union))

    # Longitud de ciclovías en un radio de 200m
    longitudes = []
    for est_geom in gdf_est.geometry:
        buffer = est_geom.buffer(200)
        total_length = 0.0
        for ciclovia in ciclovias_list:
            try:
                inter = ciclovia.intersection(buffer)
                if inter.is_empty:
                    continue
                if isinstance(inter, (LineString, MultiLineString)):
                    total_length += inter.length
            except Exception:
                continue
        longitudes.append(total_length)
    gdf_est["ciclo_len_200m"] = longitudes

    # Dejar solo columnas útiles
    df_out = gdf_est[[
        "id", "dist_ciclovia_m", "ciclo_len_200m"
    ]].rename(columns={"id": "id_estacion_origen"})

    df_out.to_csv(output_path, index=False)
    print(f"✅ Atributos de estaciones guardados en: {output_path}")

import pandas as pd
import json
import os

def final_prep(
    path_modelado: str,
    path_output: str = "data/modelado/ds_modelado_FUNCIONA.csv",
    path_atributos_estacion: str = "data/new_data/processed/atributos_estaciones.csv",
    path_barrios: str = "data/estaciones_con_barrios.csv",
    path_mapa_barrio: str = "data/modelado/mapa_barrio.json"
) -> None:
    df = pd.read_csv(path_modelado)

    df["fecha_intervalo"] = pd.to_datetime(dict(
        year=df["año_intervalo"],
        month=df["mes_intervalo"],
        day=df["dia_intervalo"],
        hour=df["hora_intervalo"],
        minute=df["minuto_intervalo"]
    ))

    fecha_min = df["fecha_intervalo"].min()
    fecha_max = df["fecha_intervalo"].max()
    intervalos = pd.date_range(start=fecha_min, end=fecha_max, freq="30min")
    estaciones = df["id_estacion_origen"].unique()

    df_regular = pd.MultiIndex.from_product(
        [estaciones, intervalos],
        names=["id_estacion_origen", "fecha_intervalo"]
    ).to_frame(index=False)

    df_lags = df[[
        "id_estacion_origen", "fecha_intervalo",
        "N_arribos_intervalo", "N_salidas_intervalo"
    ]]
    df_regular = df_regular.merge(df_lags, on=["id_estacion_origen", "fecha_intervalo"], how="left")
    df_regular[["N_arribos_intervalo", "N_salidas_intervalo"]] = df_regular[["N_arribos_intervalo", "N_salidas_intervalo"]].fillna(0)

    for lag in [1, 2, 3]:
        df_regular[f"arribos_lag{lag}"] = df_regular.groupby("id_estacion_origen")["N_arribos_intervalo"].shift(lag)
    for lag in [1, 2]:
        df_regular[f"salidas_lag{lag}"] = df_regular.groupby("id_estacion_origen")["N_salidas_intervalo"].shift(lag)

    df_regular["arribos_rolling7"] = (
        df_regular.groupby("id_estacion_origen")["N_arribos_intervalo"]
            .transform(lambda x: x.shift(1).rolling(window=7).mean())
    )
    df_regular["arribos_ultima_hora"] = (
        df_regular.groupby("id_estacion_origen")["N_arribos_intervalo"]
            .transform(lambda x: x.shift(1).rolling(window=2).sum())
    )

    lag_cols = [col for col in df_regular.columns if "lag" in col or "rolling" in col]
    df_regular[lag_cols] = df_regular[lag_cols].fillna(-1)

    df = df.merge(
        df_regular[["id_estacion_origen", "fecha_intervalo"] + lag_cols],
        on=["id_estacion_origen", "fecha_intervalo"],
        how="left"
    )

    df["fecha_origen"] = pd.to_datetime(dict(
        year=df["año_origen"],
        month=df["mes_origen"],
        day=df["dia_origen"],
        hour=df["hora_origen"],
        minute=df["minuto_origen"]
    ))
    df = df.sort_values(["id_usuario", "fecha_origen"]).copy()

    df["ultima_estacion_origen_usuario"] = df.groupby("id_usuario")["id_estacion_origen"].shift(1).fillna(-1).astype(int)
    df["ultima_estacion_destino_usuario"] = df.groupby("id_usuario")["id_estacion_destino"].shift(1).fillna(-1).astype(int)
    df["fecha_ultimo_viaje_usuario"] = df.groupby("id_usuario")["fecha_origen"].shift(1)
    df["tiempo_desde_ultimo_viaje_usuario"] = (
        (df["fecha_origen"] - df["fecha_ultimo_viaje_usuario"]).dt.total_seconds() / 60
    ).fillna(-1)

    df_barrios = pd.read_csv(path_barrios)
    with open(path_mapa_barrio, "r") as f:
        mapa_barrio = json.load(f)

    df = df.merge(
        df_barrios.rename(columns={
            "id_estacion": "ultima_estacion_origen_usuario",
            "barrio": "barrio_ultima_estacion_origen_usuario"
        }),
        on="ultima_estacion_origen_usuario", how="left"
    )
    df = df.merge(
        df_barrios.rename(columns={
            "id_estacion": "ultima_estacion_destino_usuario",
            "barrio": "barrio_ultima_estacion_destino_usuario"
        }),
        on="ultima_estacion_destino_usuario", how="left"
    )

    df["barrio_ultima_estacion_origen_usuario"] = df["barrio_ultima_estacion_origen_usuario"].map(mapa_barrio).fillna(-1).astype(int)
    df["barrio_ultima_estacion_destino_usuario"] = df["barrio_ultima_estacion_destino_usuario"].map(mapa_barrio).fillna(-1).astype(int)

    for lag in [1, 2, 3]:
        df[f"estacion_origen_viejo_us_lag{lag}"] = df.groupby("id_usuario")["id_estacion_origen"].shift(lag).fillna(-1).astype(int)
        df[f"estacion_destino_viejo_us_lag{lag}"] = df.groupby("id_usuario")["id_estacion_destino"].shift(lag).fillna(-1).astype(int)

    df["estacion_origen_viejo_us"] = df["estacion_origen_viejo_us_lag1"]
    df["estacion_destino_viejo_us"] = df["estacion_destino_viejo_us_lag1"]

    df_atributos = pd.read_csv(path_atributos_estacion)
    df = df.merge(df_atributos, on="id_estacion_origen", how="left")

    df.to_csv(path_output, index=False)
    print(f"✅ Dataset final guardado en {path_output}")

import pandas as pd
import os

def limpiar_y_guardar_parquet(
    path_csv: str,
    path_parquet: str
) -> None:
    """
    Limpia columnas innecesarias y guarda el dataset en formato Parquet.
    
    Parámetros:
    - path_csv: ruta al CSV de entrada
    - path_parquet: ruta de salida en formato .parquet
    """
    if not os.path.exists(path_csv):
        print(f"❌ No se encontró el archivo: {path_csv}")
        return

    df = pd.read_csv(path_csv)

    # Eliminar columnas innecesarias si existen
    df = df.drop(columns=[
        "lat_x", "lon_x", "lat_y", "lon_y", "fecha_ultimo_viaje_usuario"
    ], errors="ignore")

    # Rellenar columnas de ciclovías
    df["ciclo_len_200m"] = df["ciclo_len_200m"].fillna(0.0)
    df["dist_ciclovia_m"] = df["dist_ciclovia_m"].fillna(9999.0)

    # Guardar
    df.to_parquet(path_parquet, index=False)
    print(f"✅ Parquet guardado en: {path_parquet}")



# PARA LAS FUNC

In [ ]:
df_modelado = insumos_modelado()
df_modelado = procesar_columnas_modelado(df_modelado)
df_modelado.to_csv("data/modelado/ds_modelado.csv", index=False)
generar_atributos_estaciones()
final_prep(
    path_modelado="data/modelado/ds_modelado.csv",
    path_output="data/modelado/ds_modelado_final.csv"
)
limpiar_y_guardar_parquet(
    path_csv="data/modelado/ds_modelado_final.csv",
    path_parquet="data/modelado/ds_modelado.parquet"
)

In [ ]:
#ESTO ES TEST
df_test = insumos_modelado(test=True, path_test="data/recorridos/processed/trips_2023_pr.csv")
df_modelado_test = procesar_columnas_modelado(df_test)
df_modelado_test.to_csv("data/modelado/ds_modelado_test.csv", index=False)
final_prep(
    path_modelado="data/test/ds_modelado_test.csv",
    path_output="data/test/ds_modelado_test_final.csv"
)
limpiar_y_guardar_parquet(
    path_csv="data/test/ds_modelado_test_final.csv",
    path_parquet="data/test/ds_modelado_test.parquet"
)

In [ ]:
import os
import pandas as pd
from modeling import (
    insumos_modelado,
    procesar_columnas_modelado,
    construir_dataset_modelado_v2,
    final_prep,
    limpiar_y_guardar_parquet
)

def armar_dataset_modelado(test=False, path_test=None):
    if test:
        if not path_test or not os.path.exists(path_test):
            print("❌ Path de test inválido o no existe.")
            return

        print("\n🧪 Generando dataset de TEST...")
        df_test = insumos_modelado(test=True, path_test=path_test)
        df_modelado_test = procesar_columnas_modelado(df_test)

        path_csv = "data/test/ds_modelado_test.csv"
        path_csv_final = "data/test/ds_modelado_test_final.csv"
        path_parquet = "data/test/ds_modelado_test.parquet"
    
        df_modelado_test.to_csv(path_csv, index=False)

        final_prep(
            path_modelado=path_csv,
            path_output=path_csv_final
        )

        limpiar_y_guardar_parquet(
            path_csv=path_csv_final,
            path_parquet=path_parquet
        )
        print("✅ Dataset de test generado correctamente.")

    else:
        print("\n🧱 Generando dataset de TRAIN (años completos)...")
        df_train = insumos_modelado()
        df_modelado_train = procesar_columnas_modelado(df_train)

        path_csv = "data/modelado/ds_modelado.csv"
        path_csv_final = "data/modelado/ds_modelado_FUNCIONA.csv"
        path_parquet = "data/modelado/ds_modelado_FUNCIONA.parquet"

        df_modelado_train.to_csv(path_csv, index=False)

        final_prep(
            path_modelado=path_csv,
            path_output=path_csv_final
        )

        limpiar_y_guardar_parquet(
            path_csv=path_csv_final,
            path_parquet=path_parquet
        )
        print("✅ Dataset de entrenamiento generado correctamente.")

In [ ]:
from modeling import armar_dataset_modelado
from utils import preprocesamiento_completo
# Para entrenamiento (modo completo)
preprocesamiento_completo()

# Para test
preprocesamiento_completo(test=True, path_test="data/recorridos/processed/trips_2023_pr.csv")


armar_dataset_modelado(test=False)
armar_dataset_modelado(test=True, path_test="pathtest")